In [1]:
## Código para la PC2 IDCE 
# Nombre: Joaquin Emilio López Málaga 
# código: 20221484

In [2]:
# Iniciamos el procedimiento llamando a instalar a pandas y openpyxl (este último debido a que el archivo es un .xlsx)

In [3]:
pip install pandas openpyxl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
# LLamamos a pandas para empezar a trabajar, además de leer el archivo HDI.xlsx

In [5]:
import pandas as pd

In [6]:
DF_HDI = pd.read_excel('HDI.xlsx')

In [7]:
# La data Frame dada tiene dms filas innecesarias, por esta razón limitamos a 230 filas que es donde todavia hay data de valor 
DF_HDI = DF_HDI.iloc[:230]

In [8]:
# Eliminar las primeras 4 filas que no contienen datos útiles
DF_HDI_Limpio = DF_HDI.drop(index=[0, 1, 2, 3]).reset_index(drop=True)

In [9]:
# Establecemos los encabezados de forma adecuada 
DF_HDI_Limpio.columns = DF_HDI_Limpio.iloc[0]
DF_HDI_Limpio = DF_HDI_Limpio.drop(0).reset_index(drop=True)

In [10]:
# Renombrar las columnas con los nombres correctos basados en el archivo de Excel
column_nombres = [
    'HDI rank', 'Country', 'Human Development Index (HDI)', 'NADA',
    'Life expectancy at birth (years)', 'NADA', 'Expected years of schooling (years)', 
    'NADA', 'Mean years of schooling (years)', 'NADA', 
    'Gross national income (GNI) per capita (2017 PPP $)',
    'NADA','GNI per capita rank minus HDI rank', 'NADA','HDI rank (2020)'
]

In [11]:
# Establecemos el cambio 
DF_HDI_Limpio.columns = column_nombres

In [12]:
# Eliminamos las columnas específicas por índice: 3, 5, 7, 9, 11, 13
columns_eliminar = DF_HDI_Limpio.columns[[3, 5, 7, 9, 11, 13]]
DF_HDI_Limpio = DF_HDI_Limpio.drop(columns=columns_eliminar)

In [13]:
# Creamos una nueva columna 'Human development groups'
DF_HDI_Limpio['Human development groups'] = pd.NA

In [14]:
# Añadimos la lista de índices donde están las celdas combinadas (filas que no aportan información útil)
grupo_filas = [8, 75, 125, 170, 203]

In [15]:
# Asignar los valores de estas filas a la nueva columna y eliminar las filas
for index in grupo_filas:
    groupo_nombre = DF_HDI_Limpio.loc[index, 'Country']  
    DF_HDI_Limpio.loc[index + 1:, 'Human_development_groups'] = groupo_nombre  # Asignamos el grupo a las filas siguientes

In [16]:
# Eliminamos las filas con los títulos de grupo (filas combinadas)
DF_HDI_Limpio = DF_HDI_Limpio.drop(index=grupo_filas).reset_index(drop=True)

In [17]:
# Duplicamos 'Human development groups' en una nueva columna y hacer el mapeo numérico
DF_HDI_Limpio['Human_development_groups_num'] = DF_HDI_Limpio['Human development groups']
mapper1 = {
    'Low human development': 1, 
    'Medium human development': 2,
    'High human development': 3,
    'Very high human development': 4
}

In [18]:
# Reemplazamos los valores en la nueva columna
DF_HDI_Limpio['Human_development_groups_num'] = DF_HDI_Limpio['Human_development_groups_num'].replace(mapper1)

In [19]:
# Ahora Reemplazamos las comas por puntos en todas las columnas excepto 'Country'
columns_a_convertir = DF_HDI_Limpio.columns.drop(['Country'])

In [20]:
# Para este caso, vamos a convertir temporalmente a string y reemplazar comas por puntos, esto es debido a que nos sale error sin solo tratamos de cambiarlo 
DF_HDI_Limpio[columns_a_convertir] = DF_HDI_Limpio[columns_a_convertir].apply(lambda x: x.astype(str).str.replace(',', '.'))

In [21]:
# 7. Convertir nuevamente todas las columnas a numérico excepto 'Country'
DF_HDI_Limpio[columns_a_convertir] = DF_HDI_Limpio[columns_a_convertir].apply(pd.to_numeric, errors='coerce')

In [22]:
# 8. Eliminar las columnas 10 y 11
DF_HDI_Limpio = DF_HDI_Limpio.drop(DF_HDI_Limpio.columns[[10, 11]], axis=1)

In [23]:
DF_HDI_Limpio.dropna(how='all', inplace=True)

In [24]:
DF_HDI_Limpio

,HDI rank,Country,Human Development Index (HDI),Life expectancy at birth (years),Expected years of schooling (years),Mean years of schooling (years),Gross national income (GNI) per capita (2017 PPP $),GNI per capita rank minus HDI rank,HDI rank (2020),Human development groups
0,NaN,NaN,2021.000,2021.000000,2021.000000,2021.000000,2021.000000,2021.0,2020.0,NaN
1,NaN,VERY HIGH HUMAN DEVELOPMENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,Switzerland,0.962,83.987200,16.500299,13.859660,66933.004540,5.0,3.0,NaN
3,2.0,Norway,0.961,83.233900,18.185200,13.003630,64660.106220,6.0,1.0,NaN
4,3.0,Iceland,0.959,82.678200,19.163059,13.767170,55782.049810,11.0,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...
211,NaN,Sub-Saharan Africa,0.547,60.112467,10.267338,6.001639,3698.868914,NaN,NaN,NaN
213,NaN,Least developed countries,0.540,64.215856,10.182659,5.215320,2880.652487,NaN,NaN,NaN
214,NaN,Small island developing states,0.730,70.325547,12.422289,9.089640,16782.329527,NaN,NaN,NaN
216,NaN,Organisation for Economic Co-operation and Dev...,0.899,78.987605,16.459253,12.267755,45086.969067,NaN,NaN,NaN
